In [ ]:
import json
import logging
from ftplib import FTP
import os
import time
from datetime import timedelta
import datetime as dt
from datetime import datetime
from logging.handlers import RotatingFileHandler
from os.path import join


In [ ]:
# 1.Get JSON Variable********************************************************************************************

json_path = r'C:\Users\00220401626\Desktop\プログラム\Mount\config.json'

try:   
    with open(json_path) as json_file:
        config = json.load(json_file)

    host = config['host']
    username = config['username']
    password = config['password']
    remote_folder = config['remote_folder']
    local_folder = config['local_folder']
    log = config['log_path']
    days = config['days']
    last_time = config['last_time']
    # last_time = dt.strptime(last_time, '%Y/%m/%d %H:%M:%S')

except Exception as e:
    logging.info(f"Error: {e}")


In [ ]:
# 2.Logging Configuration*****************************************************************************************

try:
    if not os.path.exists(log):
        os.makedirs(log, exist_ok=True)

    dt_st = datetime.now()
    log_path = join(log, f"debug_{dt_st.strftime('%Y%m%d')}.log")
    log_formatter = logging.Formatter('%(asctime)s [%(levelname)s]: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
    log_handler = RotatingFileHandler(filename=log_path, maxBytes=1048576, backupCount=10, delay=True)
    log_handler.setFormatter(log_formatter)
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    logger.addHandler(log_handler)
    logging.info("Logging Configuration")

except Exception as e:
    logging.error(f"An error occurred while setting up logging:", e)


In [ ]:
# 3.Update_JSON****************************************************************************************************
def update_json_last_time(new_time):
    try:
        config['last_time'] = new_time
        json_data = json.dumps(config, indent=2)

        with open(json_path, 'w') as output_file:
            output_file.write(json_data)
    except Exception as e:
        print("An error occurred:", e)

In [ ]:
# 4.Delete_Old_Logs************************************************************************************************
def delete_old_logs(log, days):

    period_time = datetime.now() - timedelta(days=days)

    for filename in os.listdir(log):
        filepath = os.path.join(log, filename)
        if os.path.isfile(filepath) and datetime.fromtimestamp(os.path.getctime(filepath)) < period_time:
            try:
                os.remove(filepath)
                logging.info(f"Deleted log file: {filepath}")
            except Exception as e:
                logging.error(f"Error deleting file: {filepath} - {e}")   

In [ ]:
# 5.Download Ftp Folder to Local ***********************************************************************************
def download_ftp_files(host, username, password, remote_folder, local_folder, last_time):
    ftp = FTP(host)
    ftp.login(username, password)
    ftp.cwd(remote_folder)
    
    files = ftp.mlsd()
    if files:
        new_time = ''
        for file, meta in files:
            modified_time = datetime.strptime(meta['modify'], "%Y%m%d%H%M%S")
            if modified_time > last_time:
                local_file_path = os.path.join(local_folder, file)
                with open(local_file_path, 'wb') as local_file:
                    ftp.retrbinary(file, local_file.write)
                new_time = modified_time
        update_json_last_time(new_time) 
        logging.info(f'Last modified time is {new_time}')  
    else:
        logging.info('There are no new files to update')
        
    ftp.quit()

In [ ]:
def download_ftp_files(host, username, password, remote_folder, local_folder, last_time):
    try:
        ftp = FTP(host)
        ftp.login(username, password)
        ftp.cwd(remote_folder)

        files = ftp.mlsd()
        if files:
            new_time = ''
            for file, meta in files:
                modified_time = datetime.strptime(meta['modify'], "%Y%m%d%H%M%S")
                if modified_time > last_time:
                    local_file_path = os.path.join(local_folder, file)
                    with open(local_file_path, 'wb') as local_file:
                        ftp.retrbinary('RETR ' + file, local_file.write)
                    logging.info(f"Downloaded file: {file}")
                    new_time = modified_time
            update_json_last_time(new_time) 
            logging.info(f'Last modified time is {new_time}')
        else:
            logging.info('There are no new files to update')

    except Exception as e:
        logging.exception(f"An error occurred: {e}")

    finally:
        ftp.quit()

# download_ftp_files('ftp.example.com', 'username', 'password', '/remote_folder', '/local_folder', last_time=datetime(2024, 5, 16, 12, 0, 0))


In [ ]:
import os 
pathl= r'c:\Users\00220401626\Desktop\source'

for i in range(1, 1000):
    filename = f"log_{i}.csv"
    with open(os.path.join(pathl, filename), "w") as f:
        f.write(f"This is log file {i}")

In [ ]:
import os
import shutil

source = r'c:\Users\00220401626\Desktop\source'
destination =r'c:\Users\00220401626\Desktop\destination'
for filename in os.listdir(source):
    source_file_path = os.path.join(source, filename)
    if os.path.isfile(source_file_path):
        shutil.move(source_file_path, destination)

In [1]:
import os
import shutil

source = r'c:\Users\00220401626\Desktop\source'
destination = r'c:\Users\00220401626\Desktop\destination'

if not os.path.exists(destination):
    os.makedirs(destination)

for filename in os.listdir(source):
    source_file_path = os.path.join(source, filename)
    destination_file_path = os.path.join(destination, filename)
    
    if os.path.exists(destination_file_path):
        base, ext = os.path.splitext(filename)
        new_filename = f"{base}_copy{ext}"
        destination_file_path = os.path.join(destination, new_filename)
    
    shutil.move(source_file_path, destination_file_path)

print("Files moved successfully.")


In [3]:
import os 

source = r'c:\Users\00220401626\Desktop\source'
destination = r'c:\Users\00220401626\Desktop\destination'


os.remove(destination)

PermissionError: [WinError 5] アクセスが拒否されました。: 'c:\\Users\\00220401626\\Desktop\\destination'